# ChatGPT draw bounding boxs
Goal: Have Chatgpt draw bounding boxes by giving coordinates and using Pillow.

As of Febraury 2025, images are preprocessed before reaching ChatGPT.

Tried different method:
<ol>
  <li>Giving raw image</li>
  <li>Changing resolution of images to be within 512 by 512</li>
  <li>Changing CHATGPT detail low or high</li>
  <li>Multiple runs looking for a Consensus between CHATGPT instances</li>
  <li>Looking prompting for one que at time</li>
</ol>

Result: Chatgpt does not have the ability to accurately draw bounding boxes


In [13]:
#Libraries 
import os
#import openai
import psycopg2 #for database connection
import pandas as pd 
import matplotlib.pyplot as plt # for plotting
import json
import re

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored
import base64

In [9]:
# Define database
hostname = 'localhost'
# f = open("Username.txt", "r")
# username = f.read()
# password = f.read()
database = 'phish database'
username = 'postgres'
password = 'password'
port = "5432"

In [10]:
# Create Connection
try:
    connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database, port=port )
 
except:
    print("I am unable to connect to the database")

I am unable to connect to the database


In [6]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [7]:
#Getting Subject_ids and Image Urls from beta phase
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_beta as c;
"""
image_url = pd.read_sql_query(sql, connection)
unique_beta_subject_ids = image_url['subject_id'].unique() 
image_url

C:\Users\Num3\AppData\Local\Temp\ipykernel_16660\3474027985.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,image_url
0,40306831,https://panoptes-uploads.zooniverse.org/produc...
1,40306832,https://panoptes-uploads.zooniverse.org/produc...
2,40306833,https://panoptes-uploads.zooniverse.org/produc...
3,40306834,https://panoptes-uploads.zooniverse.org/produc...
4,40306835,https://panoptes-uploads.zooniverse.org/produc...
...,...,...
196,43857104,https://panoptes-uploads.zooniverse.org/produc...
197,43857812,https://panoptes-uploads.zooniverse.org/produc...
198,43857918,https://panoptes-uploads.zooniverse.org/produc...
199,43861756,https://panoptes-uploads.zooniverse.org/produc...


In [8]:
#Getting subject_ids and Image Urls from phase 1
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_phase1 as c;
"""

image_url = pd.read_sql_query(sql, connection)
unique_phase1_subject_ids = image_url['subject_id'].unique() 
image_url

C:\Users\Num3\AppData\Local\Temp\ipykernel_16660\1087174564.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,image_url
0,40306831,https://panoptes-uploads.zooniverse.org/produc...
1,40306832,https://panoptes-uploads.zooniverse.org/produc...
2,40306833,https://panoptes-uploads.zooniverse.org/produc...
3,40306834,https://panoptes-uploads.zooniverse.org/produc...
4,40306835,https://panoptes-uploads.zooniverse.org/produc...
...,...,...
2121,52847471,https://panoptes-uploads.zooniverse.org/produc...
2122,52847473,https://panoptes-uploads.zooniverse.org/produc...
2123,52847474,https://panoptes-uploads.zooniverse.org/produc...
2124,52847476,https://panoptes-uploads.zooniverse.org/produc...


In [ ]:
#Geting subject_id (beta image id) and subject_id_ph1 to match to urls
sql = "select subject_id, subject_id_ph1 from cybertrust_zooniverse_datamatch;"

gold_standard = pd.read_sql_query(sql, connection)
unique_gold_subject_ids = gold_standard['subject_id'].unique() 
gold_standard

C:\Users\Num3\AppData\Local\Temp\ipykernel_16660\2177744006.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,subject_id_ph1
0,52837250,52837250.0
1,52837251,52837251.0
2,52837253,52837253.0
3,52837254,52837254.0
4,52837255,52837255.0
...,...,...
1990,52847471,52847471.0
1991,52847473,52847473.0
1992,52847474,52847474.0
1993,52847476,52847476.0


In [10]:
# Dropping duplicate images that were used in the beta phase, but had a different entry for phase 1
gold_standard =gold_standard.dropna(subset=["subject_id_ph1"])
unique_gold_subject_ids = gold_standard['subject_id'].unique() 

# Checking
gold_standard

,subject_id,subject_id_ph1
0,52837250,52837250.0
1,52837251,52837251.0
2,52837253,52837253.0
3,52837254,52837254.0
4,52837255,52837255.0
...,...,...
1990,52847471,52847471.0
1991,52847473,52847473.0
1992,52847474,52847474.0
1993,52847476,52847476.0


In [11]:
#Getting Image Malicious Values
sql = """select subject_id, malicious
    from cybertrust_zooniverse_datamatch
"""

maliciousness = pd.read_sql_query(sql, connection)

###unique_subject_ids = maliciousness['subject_id'].unique() 
unique_datamatch_subject_ids = maliciousness['subject_id'].unique() 
maliciousness

C:\Users\Num3\AppData\Local\Temp\ipykernel_16660\2713104698.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,malicious
0,52837250,True
1,52837251,True
2,52837253,True
3,52837254,True
4,52837255,True
...,...,...
1990,52847471,False
1991,52847473,False
1992,52847474,False
1993,52847476,False


In [12]:
# Changing Subject_id from varying to int
change_type ={"subject_id": "int64"}
image_url = image_url.astype(change_type)
data_types = image_url.dtypes
data_types

# Merging the tables
match = pd.merge(image_url, maliciousness, on="subject_id")
unique_match_subject_ids = match['subject_id'].unique() 
match  

Prompt_list = []

In [17]:

image_path=f"512by312.jpg"

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
                                                          
base64_image = encode_image(image_path)


In [ ]:
from openai import OpenAI

# Code to Access ChatGPT
client = OpenAI(
    api_key=""
) 

#url = match['image_url'][0]
Prompt = [
        {
            "role": "system",
            "content": """You are a Cybersecurity Professional tasked with determining whether the image provided to you is a Phishing Image or not. 
            Give the information as a json object with the categories: Phishy as a strings, phishy cues as dictionaries. Inside each phishy cue dictionary have 
            coordinates as a list of intergers, and a description as a string. This image is 512px by 312px.
            """
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        "detail": "high"
                    }
                    
                },
                {
                    "type": "text",
                    "text": """Please determine whether you think the image of the website or Email is Phishing Image or Not.
                               If you believe the image is Phishy say "Something's Phishy", if not say "Nothing Phishy Here". 
                               If you believe the image is Phishy, then tell me what Phishy cues do you think are present in this list:
                               - Invalid Domain or Sender
                               - Poor Spelling or Grammar
                               - Appeal to Action - Urgency 
                               - Appeal to Action - Authority
                               - Potentially Malicious Link
                               - Other Phishy Findings
                               Please provide the coordinate system for the bounding boxes using the Pillow library use x1,y1,x2,y2. 
                               If you think a cue is located in multiple places, please provide the 
                               coordinates for multiple bounding boxes. Describe what you are putting the bounding box around."""
                },
            ]
        }
    ]
    
    #Appeal to Action - Greed
    
chat_completion = client.chat.completions.create(
    messages = Prompt,
    model= "gpt-4o",
    max_tokens=16383
    )

In [ ]:
print(chat_completion.choices[0].message.content)

```json
{
  "Phishy": "Something's Phishy",
  "phishy cues": {
    "Appeal to Action - Urgency": {
      "coordinates": [10, 15, 250, 50],
      "description": "Text offering iPhone 11 for $0 with a switch, creating urgency."
    },
    "Appeal to Action - Authority": {
      "coordinates": [10, 15, 250, 50],
      "description": "Use of AT&T name to instill trust and authority."
    },
    "Potentially Malicious Link": {
      "coordinates": [10, 50, 150, 80],
      "description": "Button encouraging clicking without verifying link."
    }
  }
}
```


In [6]:

from PIL import Image, ImageDraw
from io import BytesIO


im = Image.open("512by312.jpg")

draw = ImageDraw.Draw(im)

# Define bounding boxes with associated tags
bounding_boxes = [
    # {"box":(50, 60, 460, 150), "tag": "Appeal to Action - Greed"}, #Offer of iPhone 11
    #{"box":(20, 60, 320, 180), "tag": "Appeal to Action - Greed"}, #Offer of iPhone 11
    #{"box":(53, 60, 241, 167), "tag": "Appeal to Action - Greed"}, #Offer of iPhone 11
    #{"box":(14, 90, 260, 160), "tag": "Appeal to Action - Greed"}, #Offer of iPhone 11
    #{"box":(103, 72, 420, 144), "tag": "Appeal to Action - Authority"}, #Offer of iPhone 11
    #{"box":(10, 15, 250, 50), "tag": "Appeal to Action - Authority"}, #AT&T name to instill trust
    #{"box":(10, 15, 250, 50), "tag": "Appeal to Action - Urgency"}, #Offer of iPhone 11
    {"box":(14, 140, 100, 160), "tag": "Potentially Malicious Link"}, #Shop now button
    #{"box":(103, 180, 420, 210), "tag": "Potentially Malicious Link"}, #Shop now button
    #{"box":(10, 50, 150, 80), "tag": "Potentially Malicious Link"}, #Button encouraging clicking without verifying link
]
# Define a mapping of tags to colors
tag_colors = {
    "Invalid Domain or Sender": (255, 0, 0),  # Red
    "Poor Spelling or Grammar": (0, 255, 0),  # Green
    "Appeal to Action - Greed": (0, 0, 255),  # Blue
    "Appeal to Action - Urgency": (255, 255, 0),  # Yellow
    "Appeal to Action - Authority": (0, 255, 255),  # Cyan
    "Potentially Malicious Link": (255, 165, 0),  # Orange
    "Other Phishy Findings": (128, 0, 128)  # Purple
}

# Set the thickness of the bounding box outline
box_thickness = 5

# Draw bounding boxes with colors based on tags
for item in bounding_boxes:
    box = item["box"]
    tag = item["tag"]
    box_color = tag_colors.get(tag, (0, 0, 0))  # Default to black if tag is not found
    draw.rectangle(box, outline=box_color, width=box_thickness)

# Show the image
im.show()

In [ ]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO

# Sample image URL and match data (replace with actual match data)
url = match['image_url'][1]

response = requests.get(url)

im = Image.open(BytesIO(response.content))
resize_height = 1920/1260
resize_width = 1170/768

draw = ImageDraw.Draw(im)

# Define bounding boxes with associated tags
bounding_boxes = [
    {"box": (18  *resize_width, 15 *resize_height, 200 *resize_width, 50 *resize_height), "tag": "Invalid Domain or Sender"},
    {"box": (800 *resize_width, 100 *resize_height, 1200 *resize_width, 150 *resize_height), "tag": "Potentially Malicious Link"}
]

# Define a mapping of tags to colors
tag_colors = {
    "Invalid Domain or Sender": (255, 0, 0),  # Red
    "Poor Spelling or Grammar": (0, 255, 0),  # Green
    "Appeal to Action - Greed": (0, 0, 255),  # Blue
    "Appeal to Action - Urgency": (255, 255, 0),  # Yellow
    "Appeal to Action - Authority": (0, 255, 255),  # Cyan
    "Potentially Malicious Link": (255, 165, 0),  # Orange
    "Other Phishy Findings": (128, 0, 128)  # Purple
}

# Set the thickness of the bounding box outline
box_thickness = 5

# Draw bounding boxes with colors based on tags
for item in bounding_boxes:
    box = item["box"]
    tag = item["tag"]
    box_color = tag_colors.get(tag, (0, 0, 0))  # Default to black if tag is not found
    draw.rectangle(box, outline=box_color, width=box_thickness)

# Show the image
im.show()

In [15]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO

# Sample image URL and match data (replace with actual match data)
url = match['image_url'][2]

response = requests.get(url)

im = Image.open(BytesIO(response.content))
resize_height = 1920/1260
resize_width = 1170/768

draw = ImageDraw.Draw(im)

# Define bounding boxes with associated tags
bounding_boxes = [
    {"box": (350 *resize_width, 230 *resize_height, 910 *resize_width, 270 *resize_height), "tag": "Appeal to Action - Greed"},
    {"box": (660 *resize_width, 280 *resize_height, 780 *resize_width, 320 *resize_height), "tag": "Appeal to Action - Greed"},
    {"box": (360 *resize_width, 570 *resize_height, 550 *resize_width, 680 *resize_height), "tag": "Other Phishy Findings"}
]

# Define a mapping of tags to colors
tag_colors = {
    "Invalid Domain or Sender": (255, 0, 0),  # Red
    "Poor Spelling or Grammar": (0, 255, 0),  # Green
    "Appeal to Action - Greed": (0, 0, 255),  # Blue
    "Appeal to Action - Urgency": (255, 255, 0),  # Yellow
    "Appeal to Action - Authority": (0, 255, 255),  # Cyan
    "Potentially Malicious Link": (255, 165, 0),  # Orange
    "Other Phishy Findings": (128, 0, 128)  # Purple
}

# Set the thickness of the bounding box outline
box_thickness = 5

# Draw bounding boxes with colors based on tags
for item in bounding_boxes:
    box = item["box"]
    tag = item["tag"]
    box_color = tag_colors.get(tag, (0, 0, 0))  # Default to black if tag is not found
    draw.rectangle(box, outline=box_color, width=box_thickness)

# Show the image
im.show()